# gensim doc2vec & IMDB sentiment dataset

TODO: section on introduction & motivation

TODO: prerequisites + dependencies (statsmodels, patsy, ?)

### Requirements
Following are the dependencies for this tutorial:
    - testfixtures
    - statsmodels
    

## Load corpus

The data is small enough to be read into memory. 

In [1]:
import gensim
from gensim.models.doc2vec import TaggedDocument
from collections import namedtuple

SentimentDocument = namedtuple('SentimentDocument', 'words tags sentiment')

train_docs, test_docs = [], []  # will hold all docs in original order
for name, docs, sentiment in [
    ('train_pos', train_docs, 1.0), ('train_neg', train_docs, 0.0), ('train_unsup', train_docs, None),
    ('test_pos', test_docs, 1.0), ('test_neg', test_docs, 0.0),
]:
    with open(f'../data/imdb_sentiment/{name}.txt', encoding='utf-8') as sr:
        for i, line in enumerate(sr):
            docs.append(SentimentDocument(line.split(), [f'{name}_{i}'], sentiment))

print(f'{len(train_docs)} train docs, {len(test_docs)} test docs')

c:\program files\python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


75000 train docs, 25000 test docs


## Set-up Doc2Vec Training & Evaluation Models

Approximating experiment of Le & Mikolov ["Distributed Representations of Sentences and Documents"](http://cs.stanford.edu/~quocle/paragraph_vector.pdf), also with guidance from Mikolov's [example go.sh](https://groups.google.com/d/msg/word2vec-toolkit/Q49FIrNOQRo/J6KG8mUj45sJ):

`./word2vec -train ../alldata-id.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 0 -sample 1e-4 -threads 40 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1`

Parameter choices below vary:

* 100-dimensional vectors, as the 400d vectors of the paper don't seem to offer much benefit on this task
* similarly, frequent word subsampling seems to decrease sentiment-prediction accuracy, so it's left out
* `cbow=0` means skip-gram which is equivalent to the paper's 'PV-DBOW' mode, matched in gensim with `dm=0`
* added to that DBOW model are two DM models, one which averages context vectors (`dm_mean`) and one which concatenates them (`dm_concat`, resulting in a much larger, slower, more data-hungry model)
* a `min_count=2` saves quite a bit of model memory, discarding only words that appear in a single doc (and are thus no more expressive than the unique-to-each doc vectors themselves)

In [2]:
from gensim.models import Doc2Vec
import gensim.models.doc2vec
from collections import OrderedDict
import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

simple_models = [
    # PV-DM w/concatenation - window=5 (both sides) approximates paper's 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, size=100, window=5, negative=0, hs=1, min_count=2, workers=cores),
    # PV-DBOW 
    Doc2Vec(dm=0, size=100, negative=0, hs=1, min_count=2, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=100, window=10, negative=0, hs=1, min_count=2, workers=cores),
]

for model in simple_models:
    model.build_vocab(train_docs)
    print(model)

models_by_name = OrderedDict((str(model), model) for model in simple_models)

Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)
Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)
Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8)


Following the paper, we also evaluate models in pairs. These wrappers return the concatenation of the vectors from each model. (Only the singular models are trained.)

In [3]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[2]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[1], simple_models[0]])

## Predictive Evaluation Methods

Helper methods for evaluating error rate.

In [4]:
import numpy as np
import statsmodels.api as sm
from random import sample

# for timing
from contextlib import contextmanager
from timeit import default_timer
import time 

@contextmanager
def elapsed_timer():
    start = default_timer()
    elapser = lambda: default_timer() - start
    yield lambda: elapser()
    end = default_timer()
    elapser = lambda: end-start
    
def logistic_predictor_from_data(train_targets, train_regressors):
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    #print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set, infer_steps=3, infer_alpha=0.1, infer_subsample=0.1):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets, train_regressors = zip(*[
        (doc.sentiment, test_model.docvecs[doc.tags[0]])
        for doc in train_set if doc.sentiment is not None
    ])
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_data = test_set
    if infer_subsample < 1.0:
        test_data = sample(test_data, int(infer_subsample * len(test_data)))
    test_regressors = [test_model.infer_vector(doc.words, steps=infer_steps, alpha=infer_alpha) for doc in test_data]
    test_regressors = sm.add_constant(test_regressors)
    
    # predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_data])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

## Bulk Training

Using explicit multiple-pass, alpha-reduction approach as sketched in [gensim doc2vec blog post](http://radimrehurek.com/2014/12/doc2vec-tutorial/) – with added shuffling of corpus on each pass.

Note that vector training is occurring on *all* documents of the dataset, which includes all TRAIN/TEST/DEV docs.

Evaluation of each model's sentiment-predictive power is repeated after each pass, as an error rate (lower is better), to see the rates-of-relative-improvement. The base numbers reuse the TRAIN and TEST vectors stored in the models for the logistic regression, while the _inferred_ results use newly-inferred TEST vectors. 

(On a 4-core 2.6Ghz Intel Core i7, these 20 passes training and evaluating 3 main models takes about an hour.)

In [5]:
from collections import defaultdict
best_error = defaultdict(lambda :1.0)  # to selectively-print only best errors achieved

In [6]:
from random import shuffle
import datetime

alpha, min_alpha, passes = (0.025, 0.001, 20)
alpha_delta = (alpha - min_alpha) / passes

print(f'START {datetime.datetime.now()}')

doc_list = list(train_docs)
for epoch in range(passes):
    shuffle(doc_list)  # shuffling gets best results
    
    for name, train_model in models_by_name.items():
        # train
        duration = 'na'
        train_model.alpha, train_model.min_alpha = alpha, alpha
        with elapsed_timer() as elapsed:
            if train_model in simple_models:
                train_model.train(doc_list, total_examples=model.corpus_count, epochs=1)
            duration = f'{elapsed():.1f}'
            
        # evaluate
        eval_duration = ''
        with elapsed_timer() as eval_elapsed:
            infer_err, err_count, test_count, predictor = error_rate_for_model(train_model, train_docs, test_docs)
        eval_duration = f'{eval_elapsed():.1f}'
        best_indicator = ' '
        if infer_err < best_error[name + '_inferred']:
            best_error[name + '_inferred'] = infer_err
            best_indicator = '*'
        print(f'{best_indicator}{infer_err:f} : {epoch + 1} passes : {name}_inferred {duration}s {eval_duration}s')

    print(f'completed pass {epoch + 1} at alpha {alpha:f}')
    alpha -= alpha_delta
    
print(f'END {datetime.datetime.now()}')

START 2017-04-25 19:17:23.675307
*0.286000 : 1 passes : Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)_inferred 32.2s 7.8s
*0.168000 : 1 passes : Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)_inferred 10.5s 2.4s
*0.216800 : 1 passes : Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8)_inferred 14.9s 3.2s
*0.182400 : 1 passes : dbow+dmm_inferred 0.0s 6.1s
*0.194400 : 1 passes : dbow+dmc_inferred 0.0s 11.4s
completed pass 1 at alpha 0.025000
*0.256800 : 2 passes : Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)_inferred 34.2s 7.6s
*0.146000 : 2 passes : Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)_inferred 12.1s 3.0s
 0.244000 : 2 passes : Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8)_inferred 17.1s 3.4s
*0.148800 : 2 passes : dbow+dmm_inferred 0.0s 6.6s
*0.154800 : 2 passes : dbow+dmc_inferred 0.0s 11.5s
completed pass 2 at alpha 0.023800
*0.212400 : 3 passes : Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)_inferred 36.0s 8.0s
*0.130000 : 3 passes : Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)_inferred 11.7s 2.5s
 0.218800 : 3 passes : Doc2Vec(dm/m,d100,h

## Achieved Sentiment-Prediction Accuracy

In [7]:
# print best error rates achieved
for rate, name in sorted((rate, name) for name, rate in best_error.items()):
    print(f'{rate:f} {name}')

0.116800 Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)_inferred
0.119200 dbow+dmm_inferred
0.120000 dbow+dmc_inferred
0.176400 Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8)_inferred
0.194400 Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)_inferred


In my testing, unlike the paper's report, DBOW performs best. Concatenating vectors from different models only offers a small predictive improvement. The best results I've seen are still just under 10% error rate, still a ways from the paper's 7.42%.


## Examining Results

### Are inferred vectors close to the precalculated ones?

In [8]:
doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc; re-run cell for more examples
print('for doc %d...' % doc_id)
for model in simple_models:
    inferred_docvec = model.infer_vector(train_docs[doc_id].words)
    print('%s:\n %s' % (model, model.docvecs.most_similar([inferred_docvec], topn=3)))

for doc 38355...
Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8):
 [('train_unsup_13355', 0.9153209924697876), ('train_unsup_32658', 0.44021445512771606), ('train_neg_2435', 0.41127586364746094)]
Doc2Vec(dbow,d100,hs,mc2,s0.001,t8):
 [('train_unsup_13355', 0.9604182839393616), ('train_pos_7769', 0.4150095582008362), ('train_unsup_39714', 0.4141600430011749)]
Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8):
 [('train_unsup_13355', 0.8782677054405212), ('train_pos_225', 0.4277144968509674), ('train_neg_117', 0.4054872989654541)]


(Yes, here the stored vector from 20 epochs of training is usually one of the closest to a freshly-inferred vector for the same words. Note the defaults for inference are very abbreviated – just 3 steps starting at a high alpha – and likely need tuning for other applications.)

### Do close documents seem more related than distant ones?

In [9]:
import random

doc_id = np.random.randint(simple_models[0].docvecs.count)  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.docvecs.most_similar(doc_id, topn=model.docvecs.count)  # get *all* similar documents
print(f'TARGET ({doc_id}): «{" ".join(train_docs[doc_id].words)}»\n')
print(f'SIMILAR/DISSIMILAR DOCS PER MODEL {model}:\n')
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    doc = next(doc for doc in train_docs if doc.tags[0] == sims[index][0])
    print(f'{label} {sims[index]}: «{" ".join(doc.words)}»\n')

TARGET (15622): «i couldn't give this film a bad rating or bad review for two reasons : robin williams and toni collete . the film has the potential of being a thriller and there are some slight disturbing elements that lean to the psychological which was something the film could have focused a little on . robin williams plays gabriel noon , a storytelling night time deejay who is going through personal issues : his lover moves out and gabriel is having what seems to be a case of storyteller's block . one day he receives and reads a story written by a dying 14-year old boy named pete boland ( rory culkin ) . pete tells the story of his life and the abuse he suffered at the hands of his parents . he lives with his adopted mother and social worker , donna boland ( toni collette ) . gabriel is fascinated and begins a friendship with pete , but things seem strange when gabriel attempts to meet him and discovers the possibility that pete boland may not even exist . i won't go into detail be

(Somewhat, in terms of reviewer tone, movie genre, etc... the MOST cosine-similar docs usually seem more like the TARGET than the MEDIAN or LEAST.)

### Do the word vectors show useful similarities?

In [10]:
word_models = simple_models[:]

In [11]:
import random
from IPython.display import HTML
# pick a random word with a suitable number of occurences
while True:
    word = random.choice(word_models[0].wv.index2word)
    if word_models[0].wv.vocab[word].count > 10:
        break
# or uncomment below line, to just pick a word from the relevant domain:
word = 'comedy/drama'
similars_per_model = [str(model.most_similar(word, topn=20)).replace('), ','),<br>\n') for model in word_models]
similar_table = ("<table><tr><th>" +
    "</th><th>".join([str(model) for model in word_models]) + 
    "</th></tr><tr><td>" +
    "</td><td>".join(similars_per_model) +
    "</td></tr></table>")
print("most similar words for '%s' (%d occurences)" % (word, simple_models[0].wv.vocab[word].count))
HTML(similar_table)

most similar words for 'comedy/drama' (34 occurences)


"Doc2Vec(dm/c,d100,hs,w5,mc2,s0.001,t8)","Doc2Vec(dbow,d100,hs,mc2,s0.001,t8)","Doc2Vec(dm/m,d100,hs,w10,mc2,s0.001,t8)"
"[('comedy', 0.6131592988967896), ('thriller', 0.5856417417526245), ('chiller', 0.5728679299354553), ('farce', 0.5711425542831421), ('melodrama', 0.5277600288391113), ('mystery/comedy', 0.520356297492981), ('drama', 0.5065547823905945), ('documentary', 0.4931820333003998), ('caper', 0.49190014600753784), ('whodunit', 0.477710485458374), ('drama/comedy', 0.47550463676452637), ('romp', 0.4654447138309479), ('k-drama', 0.463994562625885), ('swashbuckler', 0.46004506945610046), ('subplot', 0.4448304772377014), ('cartoon', 0.43144601583480835), ('tale', 0.4301145076751709), ('b-movie', 0.4290980100631714), ('ditty', 0.42506682872772217), ('biopic', 0.42409414052963257)]","[(""kwouk's"", 0.3979438543319702), ('cardiovascular', 0.382252961397171), (""singleton's"", 0.380048006772995), ('irrigation', 0.3782654404640198), ('argento', 0.3746173679828644), ('dementedly', 0.3717617690563202), ('seema', 0.37132272124290466), ('screwfly', 0.37029075622558594), ('rhetoric', 0.36829674243927), (""'accident"", 0.3648317754268646), ('non-ending', 0.357203871011734), ('persisting', 0.3563710153102875), ('amore', 0.356012761592865), (""jet's"", 0.3543563485145569), ('achmed', 0.349098265171051), ('ossie', 0.34883469343185425), ('mish-mash', 0.3478613495826721), ('2-3', 0.34463465213775635), ('irving', 0.3430761396884918), ('marge', 0.34153223037719727)]","[('comedy', 0.47908416390419006), ('comedy-drama', 0.46982747316360474), ('drama', 0.42911234498023987), ('regress', 0.42287036776542664), ('diversion', 0.4086240530014038), ('undiscriminating', 0.4037162661552429), (""'slice-of-life'"", 0.4026561379432678), ('thriller', 0.39599186182022095), ('farce', 0.38499534130096436), ('potboiler', 0.38408589363098145), ('kar-kui', 0.381378710269928), ('pangborn', 0.37826529145240784), ('caper', 0.37742534279823303), ('horror/thriller', 0.37568849325180054), ('entertainer', 0.3728658854961395), ('aerospace', 0.3692465126514435), ('lv2', 0.36792242527008057), ('promos', 0.3663831949234009), ('sissle', 0.3624805510044098), ('denominator', 0.3614151179790497)]"


Do the DBOW words look meaningless? That's because the gensim DBOW model doesn't train word vectors – they remain at their random initialized values – unless you ask with the `dbow_words=1` initialization parameter. Concurrent word-training slows DBOW mode significantly, and offers little improvement (and sometimes a little worsening) of the error rate on this IMDB sentiment-prediction task. 

Words from DM models tend to show meaningfully similar words when there are many examples in the training data (as with 'plot' or 'actor'). (All DM modes inherently involve word vector training concurrent with doc vector training.)

### Are the word vectors from this dataset any good at analogies?

In [ ]:
# assuming something like
# https://word2vec.googlecode.com/svn/trunk/questions-words.txt 
# is in local directory
# note: this takes many minutes
for model in word_models:
    sections = model.accuracy('questions-words.txt')
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print('%s: %0.2f%% correct (%d of %d)' % (model, float(correct*100)/(correct+incorrect), correct, correct+incorrect))

Even though this is a tiny, domain-specific dataset, it shows some meager capability on the general word analogies – at least for the DM/concat and DM/mean models which actually train word vectors. (The untrained random-initialized words of the DBOW model of course fail miserably.)

## Slop

In [ ]:
This cell left intentionally erroneous.

To mix the Google dataset (if locally available) into the word tests...

In [ ]:
from gensim.models import KeyedVectors
w2v_g100b = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)
w2v_g100b.compact_name = 'w2v_g100b'
word_models.append(w2v_g100b)

To get copious logging output from above steps...

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
rootLogger = logging.getLogger()
rootLogger.setLevel(logging.INFO)

To auto-reload python code while developing...

In [ ]:
%load_ext autoreload
%autoreload 2